# Project 2: Red vs. Blue Encryption Method

**SUMMARY:** In this project we will be setting up the device to take in different plain text inputs, the device will encrypt these inputs and we will read back the encrypted text.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'
SS_VER = 'SS_VER_2_1'

%run "Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-base
make clean
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1


.././hal/Makefile.hal:250: *** Invalid or empty PLATFORM: NONE. Known platforms: 
+-------------------------------------------------------+
| PLATFORM      | DESCRIPTION                           |
+=======================================================+
| AVR/XMEGA Targets (8-Bit RISC)                        |
+=======================================================+
+-------------------------------------------------------+
| CWLITEXMEGA   | CW-Lite XMEGA (Alias for CW303)       |
+-------------------------------------------------------+
| CW301_AVR     | Multi-Target Board, AVR Target        |
+-------------------------------------------------------+
| CW303         | XMEGA Target (CWLite), Also works     |
|               | for CW308T-XMEGA                      |
+-------------------------------------------------------+
| CW304         | ATMega328P (NOTDUINO), Also works     |
|               | for CW308T-AVR                        |
+-----------------------------------------------

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy .././hal/stm32f0/stm32f0_startup.S -o objdir-CWNANO/stm3

In [3]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 14439 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 14439 bytes


In [4]:
from tqdm import tnrange
import aes
import numpy as np
import time

#transposes a matrix 
def transpose_matrix(lst):
    matrix = [lst[i:i + 4] for i in range(0, len(lst), 4)]
    transposed = [[matrix[j][i] for j in range(4)] for i in range(4)]
    return [elem for row in transposed for elem in row]

#Enter the system key once per program. If you want to set another key, you will have to re-program the board above

key_input = input("Enter a 16 character Key")
ky = bytearray(key_input.encode('utf-8'))

while len(ky) % 16 != 0:
    ky.append(0)
#transpose matrix for internal AES implmentation
ky_transpose = transpose_matrix(ky)
print("Set the Key to: ", [hex(num) for num in ky_transpose])
cipher = aes.aes(int.from_bytes(ky_transpose, byteorder='big'), 128)
#Send key to Nano
target.simpleserial_write('p', ky)
target.simpleserial_read('r', 16)

Enter a 16 character Key1234567891234567
Set the Key to:  ['0x31', '0x35', '0x39', '0x34', '0x32', '0x36', '0x31', '0x35', '0x33', '0x37', '0x32', '0x36', '0x34', '0x38', '0x33', '0x37']


CWbytearray(b'31 32 33 34 35 36 37 38 39 31 32 33 34 35 36 37')

In [20]:
#DO NOT RUN THIS BLOCK, THIS IS THEIR IMPLEMENTATION BEING KEPT HERE AS A REFERENCE

###After programming target, we will write several plain texts to it###
##First write the plain text, then get the encrypted text back###
from tqdm import tnrange
import aes
import numpy as np
import time

pt = bytearray([])
ct = bytearray([])

#for user input

data = input("Enter a 16 character String")
pt =bytearray(data.encode('utf-8'))


#debug plaintext and ciphertext seen in FIPS test cases
#pt = bytearray([0x32,0x88, 0x31, 0xe0, 0x43, 0x5a, 0x31, 0x37, 0xf6, 0x30, 0x98, 0x07, 0xa8, 0x8d, 0xa2, 0x34])
#ky = bytearray([0x2b, 0x28, 0xab, 0x09, 0x7e, 0xae, 0xf7, 0xcf, 0x15, 0xd2, 0x15, 0x4f, 0x16, 0xa6, 0x88, 0x3c])

#If the data is not 16 characters, pad it with zeros
while len(pt) % 16 != 0:
    pt.append(0)
print("You entered: ", data)
print("Input in Bytes: ",pt )
#iterate 16 bytes at a time through the entire message 
for i in range(0, len(pt), 16):
    pt_chunk = pt[i:i+16]
    print("Sending 16 bytes of Plaintext: ", pt_chunk)
    target.simpleserial_write('p', pt_chunk)

    #Read the ciphertext
    response = target.simpleserial_read('r', 16)
    ct[i:i+16] = transpose_matrix(response)
    print("Recieved 16 bytes of CipherText: ", response)
    #Decode the ciphertext
    plaintext = cipher.dec_once(int.from_bytes(ct[i:i+16], byteorder='big'))
    decrypted_bytes = [hex(num) for num in plaintext]
    #Print the raw bytes and the ASCII Bytes
    print("Decrytped Data: ", decrypted_bytes)
    decrypted_bytes_transpose = transpose_matrix(decrypted_bytes)
    print("Decrypted ASCII ", ''.join(chr(int(num, 16)) for num in decrypted_bytes_transpose))



    

Enter a 16 character String1234567891234567
You entered:  1234567891234567
Input in Bytes:  bytearray(b'1234567891234567')
Sending 16 bytes of Plaintext:  bytearray(b'1234567891234567')
Recieved 16 bytes of CipherText:  CWbytearray(b'12 a8 11 ce 46 8c f9 2f b7 ad 45 95 c0 d1 de 45')
Decrytped Data:  ['0x31', '0x35', '0x39', '0x34', '0x32', '0x36', '0x31', '0x35', '0x33', '0x37', '0x32', '0x36', '0x34', '0x38', '0x33', '0x37']
Decrypted ASCII  1234567891234567


In [8]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='repeat setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [12]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct
import aes
import numpy as np
import time

pt = bytearray([])
ct = bytearray([])

scope.io.clkout = 7.5E6
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()

g_step = 1

gc.set_global_step(g_step)
gc.set_range("repeat", 0, 15)
gc.set_range("ext_offset", 0, 450)
scope.glitch.repeat = 0


data = input("Enter a 16 character String")
pt =bytearray(data.encode('utf-8'))

print(pt)

reboot_flush()
sample_size = 1
for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    successes = 0
    resets = 0
    for i in range(3):
        target.flush()
            
        scope.arm()
        
        #Do glitch loop, the goal here is to see if we can get the program
        #to exit an AES round early
        target.simpleserial_write('p', pt)
        
        ret = scope.capture()
        
        #If this read returns anything but the 
        val = target.simpleserial_read_witherrors('r', 16, glitch_timeout=10)#For loop check
        
        if ret:
            print('Timeout - no trigger')
            gc.add("reset")
            resets += 1

            #Device is slow to boot?
            reboot_flush()

        else:
            if val['valid'] is False:
                reboot_flush()
                gc.add("reset")
                resets += 1
            else:
                gcnt = struct.unpack("<I", val['payload'][:4])[0]
                
                ct[0:16] = transpose_matrix(val['payload'])


                plaintext = cipher.dec_once(int.from_bytes(ct[0:16], byteorder='big'))
                decrypted_bytes = [hex(num) for num in plaintext]
                decrypted_bytes_transpose = transpose_matrix(decrypted_bytes)
                decrypt_data = ''.join(chr(int(num, 16)) for num in decrypted_bytes_transpose)
                if(data == decrypt_data):
                    gc.add("normal")
                    resets += 1
                else:
                    #Print the raw bytes and the ASCII Bytes
                    gc.add("success")
                    print("Recieved 16 bytes of CipherText: ", ct[0:16])
                    print("Decrytped Data: ", decrypted_bytes)
                    print("Decrypted ASCII ", ''.join(chr(int(num, 16)) for num in decrypted_bytes_transpose))


                
#                 if gcnt != 2500: #for loop check
#                     gc.add("success")
#                     print(gcnt)
#                     successes += 1
#                 else:
#                     gc.add("normal")
    if successes > 0:                
        print("successes = {}, resets = {}, repeat = {}, ext_offset = {}".format(successes, resets, scope.glitch.repeat, scope.glitch.ext_offset))
print("Done glitching")

Enter a 16 character String1234567891234567
bytearray(b'1234567891234567')
Recieved 16 bytes of CipherText:  bytearray(b'1594261537264837')
Decrytped Data:  ['0x60', '0xb8', '0xa6', '0x62', '0x36', '0x15', '0x3d', '0xa4', '0x13', '0x8f', '0xbb', '0xf', '0xf', '0xa9', '0x58', '0x20']
Decrypted ASCII  `6¸©¦=»Xb¤ 
Recieved 16 bytes of CipherText:  bytearray(b'1594261537264837')
Decrytped Data:  ['0x60', '0xb8', '0xa6', '0x62', '0x36', '0x15', '0x3d', '0xa4', '0x13', '0x8f', '0xbb', '0xf', '0xf', '0xa9', '0x58', '0x20']
Decrypted ASCII  `6¸©¦=»Xb¤ 


KeyboardInterrupt: 